# LLM reasoning pop quiz

Do open-sourced LLMs have the reasoning prowess of their closed-sourced siblings?

* Google Colab notebook information
  * GPU: A100-SXM 40 GB
  * Disk: 166.8 GB
* Details on flan-t5-xxl
  * Documentation: [Hugging Face model card](https://huggingface.co/google/flan-t5-xxl)
  * Latency: See below for examples with runtimes
  * Memory footprint: Roughly  GB of GPU RAM used with fp16 precision
  * License: [Apache 2.0](https://huggingface.co/google/flan-t5-xxl)


<a target="_blank" href="https://colab.research.google.com/github/daniel-furman/LLM-reasoning-pop-quiz/blob/main/notebooks/flan-t5-xxl_core_pop_quiz.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup

In [1]:
# detailed information on the GPU

!nvidia-smi

Wed Jun  7 00:39:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/daniel-furman/LLM-reasoning-pop-quiz.git

Cloning into 'LLM-reasoning-pop-quiz'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 308 (delta 81), reused 102 (delta 43), pack-reused 165
Receiving objects: 100% (308/308), 150.47 KiB | 1.98 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [3]:
!ls

LLM-reasoning-pop-quiz	sample_data


In [4]:
# install necessary libraries
import os

os.chdir("/content/LLM-reasoning-pop-quiz")
!pip install -q -U -r requirements.txt
os.chdir("..")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [5]:
# import libraries

import transformers
import torch
import time
import yaml

# import helpers

from drf_llm_boilers import llm_boiler

In [6]:
# set the seed

transformers.set_seed(4129408)

In [7]:
# print GPU available memory

free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3)
max_memory = f"{free_in_GB-2}GB"

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
max_memory

{0: '37GB'}

## Read in the yaml config for the run

In [8]:
with open(
    "/content/LLM-reasoning-pop-quiz/configs/pop_quiz_qa_style_v1.yml", "r"
) as file:
    pop_quiz = yaml.safe_load(file)
pop_quiz

{'prompts': {'zero_shot': ["Q: A juggler has 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question.",
   "Q: Daniel is in need of a haircut. His barber works Mondays, Wednesdays, and Fridays. So, Daniel went in for a haircut on Sunday. Does this make logical sense? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question."],
  'cot_few_shot': ["Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does have now? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question. Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answe

## Load the model

In [9]:
# load google/flan-t5-xxl
# see source: https://huggingface.co/google/flan-t5-xxl#usage

# this cell will take a long time, to avoid: deploy the LLM as an API inference endpoint

model_id = "google/flan-t5-xxl"

model = llm_boiler(model_id)

Load function recognized for google/flan-t5-xxl: flan_loader
Run function recognized for google/flan-t5-xxl: flan


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
print(model.name, "\n")
print(model.tokenizer, "\n")
print(model.model, "\n")

flan 

T5Tokenizer(name_or_path='google/flan-t5-xxl', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_4

## Run the model

* For text generation options, refer to [https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline)
* Below prompts are borrowed from [https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md](https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md)

### Example 1: Zero-shot reasoning conditioned on good performance
* From https://arxiv.org/abs/2205.11916

In [ ]:
# run zero shot questions

for itr, prompt in enumerate(pop_quiz["prompts"]["zero_shot"]):
    print(f"Question 1.{itr+1}")
    print(f'Prompt: "{prompt}"\n')
    start_time = time.time()
    generated_text = model.run(
        prompt=prompt,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=256,
        temperature=1.0,
        do_sample=True,
        top_p=1.0,
        top_k=50,
        num_return_sequences=1,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")
    print(f'Text generations: "{generated_text}"\n\n')

Question 1.1
Prompt: "Q: A juggler has 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question."

<pad>The juggler has 16 golf balls * 1 / 2 = 8 golf balls. There are 8 golf balls * 0.5 = 4 blue golf balls. The answer:  4.</s>
--- 2.1005547046661377 seconds ---


Text generations: "The juggler has 16 golf balls * 1 / 2 = 8 golf balls. There are 8 golf balls * 0.5 = 4 blue golf balls. The answer: 4."


Question 1.2
Prompt: "Q: Daniel is in need of a haircut. His barber works Mondays, Wednesdays, and Fridays. So, Daniel went in for a haircut on Sunday. Does this make logical sense? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question."

<pad>Daniel has to wait until the following week to get his haircut. The

### Example 2: Chain-of-thought reasoning with few-shot examples
* From https://ai.googleblog.com/2022/05/language-models-perform-reasoning-via.html


In [ ]:
# run cot few-shot questions

for itr, prompt in enumerate(pop_quiz["prompts"]["cot_few_shot"]):
    print(f"Question 2.{itr+1}")
    print(f'Prompt: "{prompt}"\n')
    start_time = time.time()
    generated_text = model.run(
        prompt=prompt,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=256,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        num_return_sequences=1,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")
    print(f'Text generations: "{generated_text}"\n\n')

Question 2.1
Prompt: "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does have now? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question. Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11. Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question."

<pad>The cafeteria had 23 - 20 = 3 apples left after making lunch. They bought 6 + 3 = 7 apples. The answer is  7.</s>
--- 1.772247552871704 seconds ---


Text generations: "The cafeteria had 23 - 20 = 3 apples left after making lunch. They bought 6 + 3 = 7 apples. The answer is 7."


Question 2.2
Prompt: "Q: Roger has 3 children. Each of his k

### Example 3: Least to most prompting
* From https://arxiv.org/abs/2205.10625


In [ ]:
# run least to most questions

for itr, prompts in enumerate(pop_quiz["prompts"]["least_to_most"]):
    print(f"Question 3.{itr+1}")
    # Start with sub question #1
    sub_question_1 = prompts[0]
    print(f'Prompt: "{sub_question_1}"\n')

    start_time = time.time()
    res_1 = model.run(
        prompt=sub_question_1,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=256,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        num_return_sequences=1,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")
    print(f'Text generation: "{res_1}"\n')

    # Now do sub question #2 by appending answer to sub question #1
    sub_question_2 = f"{sub_question_1} {res_1} {prompts[1]}"
    print(f'Prompt: "{sub_question_2}"\n')

    start_time = time.time()
    res_2 = model.run(
        prompt=sub_question_2,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=256,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        num_return_sequences=1,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")
    print(f'Text generation: "{res_2}"\n')

Question 3.1
Prompt: "Q: It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. How long does each trip take? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question."

<pad>It takes Amy 4 / 1 = 3 minutes to slide down. So each trip takes 3 / 2 = 2 minutes. The answer:  2.</s>
--- 1.7430152893066406 seconds ---


Text generation: "It takes Amy 4 / 1 = 3 minutes to slide down. So each trip takes 3 / 2 = 2 minutes. The answer: 2."

Prompt: "Q: It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. How long does each trip take? A: Let's work this out in a step by step way to be sure we have the right answer. Respond as succinctly as possible to answer the question. It takes Amy 4 / 1 = 3 minutes to slide down. So each trip takes 3 / 2 = 2 minutes. The answer: 2. Q: The slide closes in 15 minutes. How many times can she slide before it

### Example 4: Tab-CoT

* See https://arxiv.org/abs/2305.17812

In [11]:
# run tab-cot questions

for itr, prompts in enumerate(pop_quiz["prompts"]["tab_cot"]):
    print(f"Question 4.{itr+1}")
    # Start with sub question #1
    sub_question_1 = prompts[0]
    print(f'Prompt: "{sub_question_1}"\n')

    start_time = time.time()
    res_1 = model.run(
        prompt=sub_question_1,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=256,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        num_return_sequences=1,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")
    print(f'Text generation: "{res_1}"\n')

    # Now do sub question #2 by appending answer to sub question #1
    sub_question_2 = f"{sub_question_1} {res_1} {prompts[1]}"
    print(f'Prompt: "{sub_question_2}"\n')

    start_time = time.time()
    res_2 = model.run(
        prompt=sub_question_2,
        eos_token_ids=model.tokenizer.eos_token_id,
        max_new_tokens=256,
        temperature=0.01,
        do_sample=True,
        top_p=0.92,
        top_k=50,
        num_return_sequences=1,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")
    print(f'Text generation: "{res_2}"\n')

Question 4.1
Prompt: "Jackson is planting tulips. He can fit 6 red tulips in a row and 8 blue tulips in a row. If Jackson buys 36 red tulips and 24 blue tulips, how many rows of flowers will he plant? Format the response as a completion of this table.
|step|subquestion|procedure|result|
|:---|:----------|:--------|:-----:|"

<pad>Step 1: How many red tulips will Jackson plant? Step 2: How many blue tulips will Jackson plant? Step 3: How many rows of flowers will Jackson  plant?</s>
--- 7.039231300354004 seconds ---


Text generation: "Step 1: How many red tulips will Jackson plant? Step 2: How many blue tulips will Jackson plant? Step 3: How many rows of flowers will Jackson plant?"

Prompt: "Jackson is planting tulips. He can fit 6 red tulips in a row and 8 blue tulips in a row. If Jackson buys 36 red tulips and 24 blue tulips, how many rows of flowers will he plant? Format the response as a completion of this table.
|step|subquestion|procedure|result|
|:---|:----------|:--------|:---